**Converts Showcase <a href='https://biobank.ndph.ox.ac.uk/showcase/label.cgi?id=100092'>cancer registry data</a> and HES <a href='https://biobank.ndph.ox.ac.uk/showcase/label.cgi?id=2002'>Summary Diagnoses</a> from wide (i.e. one row per participant) into long (i.e. one row per instance/event) format. This enables for easier filtering than looping through each instance column.**

In [3]:
import pandas as pd
import yaml

# Load YAML config file
with open("../config.yml", "r") as file:
    config = yaml.safe_load(file)

Cancer registry

In [ ]:
# Define cancer instanced field stub names
instanced_field_names = {
    'f.40021': 'origin',
    'f.40019': 'format',
    'f.40009': 'occurences',
    'f.40005': 'date_diagnosis',
    'f.40008': 'age_diagnosis',
    'f.40006': 'ICD10',
    'f.40013': 'ICD9',
    'f.40011': 'histology',
    'f.40012': 'behaviour',
}

# Add all instance columns of each field to a list
all_instance_field_names = []
for field in instanced_field_names:
    # Define fields with specific number of instances
    if field == 'f.40009':
        n_instances = 1
    elif field == 'f.40013':
        n_instances = 15
    else:
        n_instances = 22
    for instance in range(n_instances):
            all_instance_field_names.append(field + '.' + str(instance) + '.0')

In [ ]:
# Load all relevant column from master dataset
cancers_wide = pd.read_csv(config['paths']['ukb_showcase_raw_dataset'], sep="\t", usecols=['f.eid'] + all_instance_field_names)

In [ ]:
# Drop rows where all fields other than ID are missing
non_id_cols = cancers_wide.columns.values.tolist()
non_id_cols.remove("f.eid")
cancers_wide = cancers_wide.dropna(how='all', subset=non_id_cols)

In [47]:
# Rename cols to remove array notation as these make wide_to_long difficult (arrays not used in cancer registry data)
cancers_wide.columns = ['eid'] + [col[:-2] for col in cancers_wide.columns[1:]]

In [87]:
# Convert from wide to long format using pd function
cancers_long = pd.wide_to_long(cancers_wide, list(instanced_field_names.keys()), sep='.', i='eid', j='instance')

# Drop rows with all missing fields
cancers_long = cancers_long.dropna(how='all')

# Reset index, as wide_to_long sets index as combination of i and j
cancers_long = cancers_long.reset_index()

# Remove unnecessary 'instance' col
cancers_long = cancers_long.drop(columns='instance')

# Rename cols using verbose names
cancers_long = cancers_long.rename(columns=instanced_field_names)

In [90]:
# Save long file
cancers_long.to_csv('../data/cancers_long.csv', index=False)

HES

In [2]:
# Define all HES ICD9/10 & dates columns (all arrays)
hes_icd10_cols = [f"41270.0.{i}" for i in range(259)]
hes_icd10_date_cols = [f"41280.0.{i}" for i in range(259)]
hes_icd9_cols = [f"41271.0.{i}" for i in range(47)]
hes_icd9_date_cols = [f"41281.0.{i}" for i in range(47)]

# Load relevant columns from master dataset
hes = pd.read_feather("../data/all_showcase_baseline.feather", columns=['eid']+hes_icd10_cols+hes_icd10_date_cols+hes_icd9_cols+hes_icd9_date_cols)

In [4]:
# Convert to long format
hes_long = pd.wide_to_long(hes, stubnames=['41270.0', '41280.0', '41271.0', '41281.0'], i='eid', j='array', sep='.')

# Drop rows where all empty, split into separate ICD10 & ICD9 and concat into one large df with shared date column
hes_long = hes_long.dropna(how='all').reset_index()
ICD10 = hes_long[['eid', '41270.0', '41280.0']].rename(columns={'41270.0': 'ICD10', '41280.0': 'date'}).dropna(subset=['ICD10', 'date'], how='all')
ICD9 = hes_long[['eid', '41271.0', '41281.0']].rename(columns={'41271.0': 'ICD9', '41281.0': 'date'}).dropna(subset=['ICD9', 'date'], how='all')
hes_final = pd.concat([ICD10, ICD9])
hes_final = hes_final[['eid', 'ICD10', 'ICD9', 'date']]

# Save
hes_final.to_feather('../data/hes_long.feather')